In [1]:
%%capture
%pip install unsloth

In [4]:
from transformers import TextStreamer
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype
)

==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:  15%|#5        | 346M/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [5]:
FastLanguageModel.for_inference(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm

In [ ]:
# prompt_style = """Bạn là một giáo viên phòng tư vấn tuyển sinh đại học Bách Khoa Hà Nội. Bạn có nhiệm vụ tư vấn tuyển sinh cho người dùng và trả lời đúng những câu hỏi mà người dùng đặt ra.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# {}"""


# inputs = tokenizer(
#     [
#         prompt_style.format(
#             "Sử dụng ngữ cảnh dưới đây để trả lời câu hỏi",
#             "Context: STT 01, Mã & Tên chương trình: (BF1) Kỹ thuật Sinh học, Họ và tên: PGS. Nguyễn Lan Hương, Email: huong.nguyenlan@hust.edu.vn, Điện thoại: 0903247172.\nQuestion: Em muốn tìm hiểu về chương trình Kỹ thuật Sinh học, em có thể liên hệ với ai ạ?",
#             "",
#         )
#     ],
#     return_tensors="pt",
# ).to("cuda")

# text_streamer = TextStreamer(tokenizer)
# _ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=128)

In [6]:
hugging_face_token = "hf_EgINeczpAuYOWkOCWSQxQLTvqSRtaxOameM"

In [7]:
from huggingface_hub import login
from getpass import getpass

# Yêu cầu người dùng nhập token của Hugging Face
hf_token = getpass("Enter your Hugging Face token: ")

# Đăng nhập vào Hugging Face
login(hf_token)


Enter your Hugging Face token: ··········


In [8]:
wandb_api = "35d539b1c854e7ecdeaea8c2f6769ef332bd4594"

In [9]:
import wandb
# Yêu cầu người dùng nhập W&B API key
wb_token = getpass("Enter your Weights & Biases API key: ")

# Đăng nhập vào W&B
wandb.login(key=wb_token)

# Khởi tạo dự án trên W&B
run = wandb.init(
    project='Fine-tune Llama-3.1-8B-bnb-4bit on Math Dataset',
    job_type="training",
    anonymous="allow"
)


Enter your Weights & Biases API key: ··········


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: onion209-hust (onion209-hust-hust). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Xử lý dữ liệu

In [10]:
import pandas as pd
df1 = pd.read_excel('/content/Q&A&C.xlsx')
df1.head()

,context,question,answer
0,"STT 01, Mã & Tên chương trình: (BF1) Kỹ thuật ...",Em muốn tìm hiểu về chương trình Kỹ thuật Sinh...,"Chào em, để tìm hiểu về chương trình Kỹ thuật ..."
1,"STT 03, Mã & Tên chương trình: (BF-E12) Kỹ thu...",Cho em hỏi về chương trình Kỹ thuật Thực phẩm ...,"Em ơi, nếu em quan tâm đến chương trình Kỹ thu..."
2,"STT 06, Mã & Tên chương trình: (CH2) Hóa học, ...","Em có thắc mắc về ngành Hóa học, em có thể hỏi...","Em ơi, nếu em có bất kỳ thắc mắc nào về ngành ..."
3,"STT 07, Mã & Tên chương trình: (MS5) Kỹ thuật ...",Em muốn tìm hiểu về ngành Kỹ thuật in thì liên...,"Để tìm hiểu về ngành Kỹ thuật in (mã MS5), em ..."
4,"STT 11, Mã & Tên chương trình: (EE1) Kỹ thuật ...","Em muốn tìm hiểu về ngành Kỹ thuật Điện, em li...","Chào em, để tìm hiểu về chương trình Kỹ thuật ..."


In [11]:
df1['answer'] = df1['answer'].astype(str)

In [12]:
from datasets import Dataset
dataset_new = Dataset.from_pandas(df1)
print(dataset_new.column_names)

['context', 'question', 'answer']


In [13]:
alpaca_prompt = """Bạn là một giáo viên phòng tư vấn tuyển sinh đại học Bách Khoa Hà Nội.
 Bạn có nhiệm vụ tư vấn tuyển sinh cho người dùng và trả lời đúng những câu hỏi mà người dùng đặt ra.
 Trả lời câu hỏi một người dùng thân thiện, chính xác. Nếu không tìm được thông tin hãy trả lời 'Vui lòng tra thêm thông tin trên website: 'https://ts.hust.edu.vn/'.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["question"]
    inputs       = examples["context"]
    outputs      = examples["answer"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

dataset = dataset_new.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/1940 [00:00<?, ? examples/s]

In [14]:
print(dataset[0]['text'])

Bạn là một giáo viên phòng tư vấn tuyển sinh đại học Bách Khoa Hà Nội.
 Bạn có nhiệm vụ tư vấn tuyển sinh cho người dùng và trả lời đúng những câu hỏi mà người dùng đặt ra. 
 Trả lời câu hỏi một người dùng thân thiện, chính xác. Nếu không tìm được thông tin hãy trả lời 'Vui lòng tra thêm thông tin trên website: 'https://ts.hust.edu.vn/'.

### Instruction:
Em muốn tìm hiểu về chương trình Kỹ thuật Sinh học, em có thể liên hệ với ai ạ?

### Input:
STT 01, Mã & Tên chương trình: (BF1) Kỹ thuật Sinh học, Họ và tên: PGS. Nguyễn Lan Hương, Email: huong.nguyenlan@hust.edu.vn, Điện thoại: 0903247172

### Response:
Chào em, để tìm hiểu về chương trình Kỹ thuật Sinh học (mã BF1), em có thể liên hệ với cô Nguyễn Lan Hương qua email huong.nguyenlan@hust.edu.vn hoặc số điện thoại 0903247172 nhé.<|eot_id|>


In [15]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",

    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)


Unsloth 2025.1.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [16]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/1940 [00:00<?, ? examples/s]

In [17]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,940 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 24,313,856
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
1,2.652600
2,2.483600
3,2.531400
4,2.517700
5,2.269300
6,2.035500
7,1.905700
8,1.677400
9,1.603200
10,1.273300


In [18]:
%mkdir ../temp
%cd /kaggle/temp

[Errno 2] No such file or directory: '/kaggle/temp'
/content


In [19]:
model.push_to_hub_merged("Tuyen_sinh_model", tokenizer, save_method = "merged_16bit")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.2G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.09 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:01<00:00, 25.91it/s]


Unsloth: Saving to organization with address onionhust/Tuyen_sinh_model
Unsloth: Saving tokenizer... Done.
Unsloth: Saving to organization with address onionhust/Tuyen_sinh_model
Unsloth: Saving onionhust/Tuyen_sinh_model/pytorch_model-00001-of-00002.bin...
Unsloth: Saving onionhust/Tuyen_sinh_model/pytorch_model-00002-of-00002.bin...
Unsloth: Uploading all files... Please wait...


pytorch_model-00001-of-00002.bin:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/None/Tuyen_sinh_model
